In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
os.chdir(r"C:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection")

import pandas as pd
from resources.constants import *

outfits_df = pd.read_csv(OUTFITS_CSV_PATH, sep=';')
outfits_df.head()

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories
0,outfit.fffdaa715c3646f8b1c0f04d549ff07e,Out of stock - Asymmetric Frilled Dress,"This fun, short dress features and asymmetric ...",group.50a586c78eb7626e294ba3bd07d12c79,464,2017-12-30 11:28:01.000,4000.0,"['Metallic', 'Synthetic', 'Cotton', 'Sandro', ...","['Details', 'Material', 'Material', 'Brand', '..."
1,outfit.fffa1b9a3db6415d806f3c48f8ab58d9,Yellow Shell Mellomholmene Blouse,This beautiful blouse features an adjustable n...,group.61ad2fcabb3e9197e3836376e6b67f2c,112,2021-06-07 12:07:22.921,1300.0,"['Yellow', 'Cotton', 'Blouses', 'Everyday', 'M...","['Color', 'Material', 'Category', 'Occasion', ..."
2,outfit.fff175b13ceb453f9928625491412ede,Kaula Dress Black,Kaula from Rodebjer is a fitted dress made in ...,group.37c2b59d63d3a9c2d58e07f532f71f7f,635,2023-06-05 09:17:59.004,3100.0,"['Synthetic', 'Multi Season', 'Rodebjer', 'Eve...","['Material', 'Seasons', 'Brand', 'Occasion', '..."
3,outfit.ffef9d7c292a48b69076d2df2e32352f,For sale - Jarvis Blouse,This wrap blouse has mid length sleeves and a ...,group.dfcaa57546b0b7a5e9eb204449b6cc1c,745,2021-05-18 14:02:28.690,1500.0,"['Cotton', 'Multi Season', 'Floral', 'Wrap', '...","['Material', 'Seasons', 'Details', 'Fit', 'Siz..."
4,outfit.ffeef842238f4dbdabc6c730a75aa2bd,Black Amber Pants,"Feel slack and nice dressed with this pant, ma...",group.ee297c977905eb21a123a4aea5fbb6d2,504,2021-07-16 14:02:30.643,1200.0,"['Winter', 'Cotton', 'L', 'Knitwear', 'Everyda...","['Seasons', 'Material', 'Size', 'Category', 'O..."


In [2]:
third_chance_df = pd.read_csv(THIRD_CHANCE_CSV_PATH, sep=';')
print(third_chance_df.shape)
third_chance_df.head()

(1296, 11)


,date_added,id,name,brand,owner,condition,condition_desctription,retail_price,tc_price,sold,varetelling
0,2023-11-23,-,Blue Scarf 1,Holzweiler,Holzweiler,6 - Redesigned,Ødelagt på armene. Stygg slitasje,2500,650.0,True,False
1,2023-11-23,-,Blue Scarf 2,Holzweiler,Holzweiler,6 - Redesigned,NaN,2800,750.0,True,False
2,2023-11-23,-,Green Scarf 1,Holzweiler,Holzweiler,5 - Damaged,Slitt og går ikke an å vaskes,12000,2400.0,False,False
3,2023-11-23,-,Green Scarf 2,Holzweiler,Holzweiler,5 - Damaged,Masse flekker,10000,1800.0,False,False
4,2023-11-23,-,Green Scarf 3,Holzweiler,Holzweiler,1 - As new,tror det bare er fordi den er en singlet,500,250.0,False,True


In [4]:
third_chance_df["date_added"] = pd.to_datetime(third_chance_df["date_added"], errors='coerce', yearfirst=True)

third_chance_df = third_chance_df.dropna(subset=["date_added", "id"])
third_chance_df = third_chance_df.drop_duplicates(subset=["id"])
third_chance_df = third_chance_df.merge(outfits_df, left_on="id", right_on="id", how="inner", suffixes=(None, "_outfits"))#.drop_duplicates(subset=["id_outfits"])
third_chance_df["timeCreated"] = pd.to_datetime(third_chance_df["timeCreated"])
third_chance_df["time_diff"] = third_chance_df["date_added"] - third_chance_df["timeCreated"]
third_chance_df["time_diff"].describe()

In [6]:
import re
NUMERIC_PATTERN = re.compile(r"(\d+)")

def find_numeric_condition(x):
    if type(x) != str:
        return None

    match = NUMERIC_PATTERN.search(x)
    if match:
        return int(match.group(1))
    return None

third_chance_df["condition_num"] = third_chance_df["condition"].apply(find_numeric_condition)

In [7]:
third_chance_outfits_df = outfits_df[outfits_df["id"].isin(third_chance_df["id"])]
third_chance_outfits_df = third_chance_outfits_df.merge(third_chance_df, left_on="id", right_on="id", how="inner", validate="one_to_one", suffixes=(None, "_third_chance"))
third_chance_outfits_df.drop(third_chance_outfits_df.filter(regex='_third_chance$').columns.tolist(), axis=1, inplace=True)

In [8]:
third_chance_outfits_df[third_chance_outfits_df["condition_num"] >= 3]

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,date_added,...,condition,condition_desctription,retail_price,tc_price,sold,varetelling,name_outfits,owner_outfits,time_diff,condition_num
72,outfit.e9754e1dba324755a1e327c3105fd036,"The Long Sleeved Kaftan Dress, Black",Say hello to the Long Sleeved Kaftan - our mos...,group.de3bd07fdee460d07dbe02df9a5043d6,587,2021-09-14 13:32:40.966,1800.0,"['AWAN', 'Everyday', 'Fall', 'Dresses', 'XS', ...","['Brand', 'Occasion', 'Seasons', 'Category', '...",2023-12-12,...,4 - Minor defects,Stains,4700,1500.0,False,False,"The Long Sleeved Kaftan Dress, Black",587,818 days 10:27:19.034000,4.0
73,outfit.e965f8f929a345ecab825313cd4b3f24,S1 Gardenia Elysian Shirt,An elegant and feminine blouse with ruffle det...,group.4df8a3e68d3bf1b1fb825e729f2acd44,248,2020-07-03 10:08:04.005,1200.0,"['Ruffles', 'Viscose', 'Multi Season', 'Everyd...","['Details', 'Material', 'Seasons', 'Occasion',...",2023-12-12,...,4 - Minor defects,Wear and tear,1400,400.0,False,False,S1 Gardenia Elysian Shirt,248,1256 days 13:51:55.995000,4.0
74,outfit.e94be40cf46643c59bc777a9cb3f58fa,1 Cameron Dotted Blouse,"This delicate dotted blouse comes in a soft, s...",group.0b94d4e8c8c93842903d175da87331c2,745,2020-10-15 13:05:23.184,2000.0,"['Ruffles', 'Multi Season', 'Everyday', 'Ganni...","['Details', 'Seasons', 'Occasion', 'Brand', 'S...",2024-01-25,...,4 - Minor defects,Wear and tear,600,250.0,False,False,1 Cameron Dotted Blouse,745,1196 days 10:54:36.816000,4.0
75,outfit.e9319dccf9234668a6951834dcaff863,Isabella Wrap Top Mint Dandy,"The Isabella wrap top features a peplum waist,...",group.2780c47a10e8be907908012828c7afd5,249,2019-02-05 13:43:11.000,2700.0,"['Silk', 'Winter', 'Floral', 'Everyday', 'Fall...","['Material', 'Seasons', 'Details', 'Occasion',...",2022-09-26,...,4 - Minor defects,Wear and tear,1700,500.0,False,False,Isabella Wrap Top Mint Dandy,249,1328 days 10:16:49,4.0
76,outfit.e8bc2a3b91ad4a7c8745b2eb4031dd09,Marian Swirl Top Raw Umbra,Marian Swirl is a polo neck in a luxurious vis...,group.19c32f71494778d391423fdf86b661c2,416,2022-07-19 09:55:06.673,1800.0,"['Winter', 'Viscose', 'Rodebjer', 'Everyday', ...","['Seasons', 'Material', 'Brand', 'Occasion', '...",2024-01-25,...,3 - Good condition,NaN,4650,1400.0,False,False,Marian Swirl Top Raw Umbra,416,554 days 14:04:53.327000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,outfit.01738c87529546bfb8c032dc16027359,Shoulder Embellished Tunic,The Platinum Shoulder Embellished Tunic in Pla...,group.e0f2a30dd908deca2960f51ba031890e,435,2022-05-23 14:30:01.014,2700.0,"['Silk', 'Dressed-up', 'Midi', 'Everyday', 'Dr...","['Material', 'Occasion', 'Length', 'Occasion',...",2023-05-15,...,5 - Damaged,NaN,1700,340.0,False,False,Shoulder Embellished Tunic,435,356 days 09:29:58.986000,5.0
979,outfit.0118b605888840348f80965c2d8865cf,Wabi Embroidered Bright White Shirt,The FWSS Wabi Embroidery Shirt is a crisp 100%...,group.92a5ab7fdf4cc111e8f4ae277d9cd013,592,2022-06-22 04:12:07.947,2000.0,"['Dressed-up', 'Cotton', 'Everyday', 'XS', 'Su...","['Occasion', 'Material', 'Occasion', 'Size', '...",2023-01-10,...,5 - Damaged,NaN,NaN,NaN,False,False,Wabi Embroidered Bright White Shirt,592,201 days 19:47:52.053000,5.0
980,outfit.0113e4422a3f408f8fc9a94d2861474f,Sand Poddle99 Blazer,This regular fitted blazer features two button...,group.1a44560b56290e6039f49dc083645a0e,194,2021-08-27 12:30:04.490,3500.0,"['Synthetic', 'L', 'Multi Season', 'Holzweiler...","['Material', 'Size', 'Seasons', 'Brand', 'Occa...",2022-08-18,...,5 - Damaged,Lite Hull både foran og bak. Kan sys? eventuel...,2400,300.0,False,True,Sand Poddle99 Blazer,194,355 days 11:29:55.510000,5.0
981,outfit.0106cbd7e23f41da92e3081146b4ad03,Juliette Skirt Red Line,"The Juliette skirt is the perfect, flowy, ligh...",group.6c4aa786f2b635e75d1c9853dfaa47fe,249,2019-02-19 10:00:52.000,1300.0,"['Synthetic', 'Multi Season', 'Midi', 'Everyda...","['Material'

In [9]:
triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
triplets_df.head()

,customer.id,outfit.id,meta.validFrom,derived.bookingTime
0,969,outfit.5c081909537b42239e465d2d615c705f,2023-04-27 12:46:35.184,2023-03-19 14:24:09.539
1,209,outfit.c34969dd8b334064aa90bfb60c8ec308,2023-05-05 09:44:38.200,2023-01-17 13:36:53.371
2,209,outfit.aef4cc93eebf40ca8820790deb7a8323,2023-05-05 09:44:38.200,2023-01-17 13:36:53.371
3,2235,outfit.3de5df48a14b4a9aba6d8e41d11e9351,2022-11-22 03:04:56.144,2021-10-04 10:00:17.854
4,2235,outfit.0eaa358af14e469894062591bd42f38b,2022-11-22 03:04:56.144,2021-10-04 10:00:17.854


In [10]:
triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
rentals_count_dict = triplets_df["outfit.id"].value_counts().to_dict()
def get_rentals_count(x):
    return rentals_count_dict.get(x, 0)

third_chance_outfits_df["rentals_count"] = third_chance_outfits_df["id"].map(get_rentals_count)

In [11]:
third_chance_outfits_df[["time_diff", "condition_num", "rentals_count"]].corr()

,time_diff,condition_num,rentals_count
time_diff,1.000000,-0.193208,0.470471
condition_num,-0.193208,1.000000,-0.021340
rentals_count,0.470471,-0.021340,1.000000


In [12]:
import ast

def convert_string_to_list(x):
    if type(x) != str:
        return []
    return ast.literal_eval(x)

third_chance_outfits_df["outfit_tags"] = third_chance_outfits_df["outfit_tags"].apply(convert_string_to_list)
third_chance_outfits_df["tag_categories"] = third_chance_outfits_df["tag_categories"].apply(convert_string_to_list)

In [13]:
import numpy as np

def get_outfit_category(tag_categories, tags, category):
    tag_categories, tags = np.array(tag_categories), np.array(tags)
    category_indexes = np.where(tag_categories == category)[0]
    if len(category_indexes) == 0:
        return []
    cat_tags = tags[category_indexes]
    return cat_tags
    
    # try:
    #     cat_index = tag_categories.index(category)
    #     print(cat_index, len(tags), tag_categories, tags)
    #     return tags[cat_index]
    # except ValueError:
    #     return None

third_chance_outfits_df["brand"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Brand"), axis=1)
third_chance_outfits_df["material"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Material"), axis=1)
third_chance_outfits_df["size"] = third_chance_outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Size"), axis=1)

In [15]:
third_chance_outfits_df.sort_values(by="group")

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,date_added,...,tc_price,sold,varetelling,name_outfits,owner_outfits,time_diff,condition_num,rentals_count,material,size
421,outfit.93dcb93c8c104b50bcfe3d83b36e1d53,Prenestina Jacket Baby Blue Jacket,"Super elegant, constructed jacket with puffed ...",group.000aad5ac8cfefd8063ee16f6f57b2c1,101,2022-03-28 13:54:38.895,2500.0,"[Cotton, C.L.E.A.N by Christina Ledang, Viscos...","[Material, Brand, Material, Size, Category, Oc...",2023-05-15,...,849.0,False,True,Prenestina Jacket Baby Blue Jacket,101,412 days 10:05:21.105000,2.0,0,"[Cotton, Viscose]",[L]
58,outfit.edbe38fccd174c3cac918d1347b7811e,Prenestina Jacket Baby Blue Jacket,"Super elegant, constructed jacket with puffed ...",group.000aad5ac8cfefd8063ee16f6f57b2c1,101,2022-03-28 13:54:38.895,2500.0,"[Cotton, C.L.E.A.N by Christina Ledang, Viscos...","[Material, Brand, Material, Category, Occasion...",2023-01-10,...,650.0,False,False,Prenestina Jacket Baby Blue Jacket,101,287 days 10:05:21.105000,2.0,3,"[Cotton, Viscose]",[S]
464,outfit.8b5234b3619cf5fb,Hilde Manor Top,This long-sleeved top with luxurious lace embr...,group.00ea09a37a420dbd3b02b973f5fc43c4,249,2017-08-22 16:10:53.000,1800.0,"[Synthetic, Multi Season, Everyday, S, FWSS, T...","[Material, Seasons, Occasion, Size, Brand, Cat...",2024-01-25,...,549.0,True,False,Hilde Manor Top,249,2346 days 07:49:07,2.0,6,[Synthetic],[S]
808,outfit.334b32157287403aab73d9cc978a93dd,Bright White Torunn Top,"The Torunn top features embroidered hems, a co...",group.010dfb6d1c3b9a18cbcceb1aa3547b84,252,2020-09-25 10:12:54.755,2500.0,"[Cotton, Blouses, Everyday, XS, Summer, White,...","[Material, Category, Occasion, Size, Seasons, ...",2022-09-15,...,800.0,True,True,Bright White Torunn Top,252,719 days 13:47:05.245000,5.0,9,[Cotton],[XS]
438,outfit.90acfab765674a3ea8382b104aba70af,Feather Embroidery Front Pleated Dress,The Feather Embroidery Front Pleated Dress in ...,group.010f6372ca76731e871e244041faa3e2,435,2022-05-23 14:30:01.014,1900.0,"[Silk, Dressed-up, Cotton, Midi, Everyday, Dre...","[Material, Occasion, Material, Length, Occasio...",2023-09-19,...,599.0,False,False,Feather Embroidery Front Pleated Dress,435,483 days 09:29:58.986000,2.0,0,"[Silk, Cotton]",[S]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,outfit.c9b02cffa22042118df20e0d722f6fd5,Sierra Dress Riviera blue,Sierra from ROTATE is a long-sleeved jersey dr...,group.fe5c56eb2d396a69f72bf761729c3fa2,348,2023-04-20 12:00:48.382,2200.0,"[Metallic, Synthetic, Multi Season, Midi, Dres...","[Details, Material, Seasons, Length, Category,...",2023-12-12,...,3000.0,True,False,Sierra Dress Riviera blue,348,235 days 11:59:11.618000,NaN,3,[Synthetic],[S]
524,outfit.7cff339e7950448c8485a2b44205adc0,Bellix Night Blue Blouse,Look simple and luxurious in this silk top. Fe...,group.fe65c72d7049e270ca0242552ecb41e3,249,2018-11-30 12:28:06.000,3000.0,"[Silk, Multi Season, Everyday, S, Tops, By Mal...","[Material, Seasons, Occasion, Size, Category, ...",2022-09-26,...,1200.0,True,False,Bellix Night Blue Blouse,249,1395 days 11:31:54,2.0,8,[Silk],[S]
365,outfit.9e8dd974e09f44ef886fce0d3557d652,Bellix Night Blue Blouse,Look simple and luxurious in this silk top. Fe...,group.fe65c72d7049e270ca0242552ecb41e3,249,2018-11-30 12:23:43.000,3000.0,"[Silk, Everyday, Fall, XL, Spring, Tops, By Ma...","[Material, Occasion, Seasons, Size, Seasons, C...",2022-09-15,...,200.0,False,True,Bellix Night Blue Blouse,249,1384 days 11:36:17,2.0,6,[Silk],[XL]
101,outfit.e350fdafca034bd3b311d288a391ad41,Isadora Top Lime Green Floral,The Isadora top by Faithful The Brand in Luda ...,group.fe69bb88097e3a80570800493b044ec4,416,2022-06-28 14:57:23.167,1300.0,"[Ruffles, Synthetic, Purple, L, Floral, Everyd...","[Details, Material, Color, Size, Details, Occa...",2024-01-25,...,1400.0,False,False,Isadora Top Lime Green Floral,416,575 days 09:02:36.833000,3.0,0,[Synthetic],[L]


In [13]:
all_brand_classes = third_chance_outfits_df["brand"].explode()
mlb_brand_classes = all_brand_classes.value_counts()[all_brand_classes.value_counts() > 10].index.to_list()

all_material_classes = third_chance_outfits_df["material"].explode()
mlb_material_classes = all_material_classes.value_counts()[all_material_classes.value_counts() > 10].index.to_list()

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

brand_mlb = MultiLabelBinarizer(classes=mlb_brand_classes)
brand_df = pd.DataFrame(brand_mlb.fit_transform(third_chance_outfits_df["brand"]), columns=["brand_" + mlb_class for mlb_class in brand_mlb.classes_],  index=third_chance_outfits_df.index)

material_mlb = MultiLabelBinarizer()
material_df = pd.DataFrame(material_mlb.fit_transform(third_chance_outfits_df["material"]), columns=["material_" + mlb_class for mlb_class in material_mlb.classes_],  index=third_chance_outfits_df.index)

c:\Users\kaborg15\AppData\Local\anaconda3\envs\baseline-env\lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['2NDDAY', 'Acne Studios', 'Adrianna Papell', 'Alaïa', 'Alexander McQueen', 'Altuzarra', 'Ami Amalia', 'Ane Mone', 'Aris', 'Armani', 'Asos', 'Balenciaga', 'Balmain', 'Barbour', 'Bastet Noir', 'Baum und Pferdgarten', 'Billie & me', 'Britt Sisseck', 'Burberry', 'By Malina', 'Cala Jade', 'Camilla Pihl', 'Cardsome', 'Carin Wester', 'Carven', 'Cathrine Hammel', 'Celine', 'Chanel', 'Chloé', 'Christina Ledang', 'Color Vision', 'ConfettiBird', 'Creative Collective', 'Dale of Norway', 'Day Birger et Mikkelsen', 'Designers Remix', 'Diana Orving', 'Diane von Furstenberg', 'Dolce & Gabbana', 'Envelope1976', 'Filippa K', 'Five Units', 'Fleischer Couture', 'Forte Forte', 'Givenchy', 'Gucci', 'H&M', 'H20Fagerholt', 'Hekne', 'Hofmann Copenhagen', 'Houdini', 'Hunkydory', 'IBEN', 'ILAG', 'IRO', 'Ida Sjöstedt', 'InWear', 'Jorun x Vibrent', 'Karen By Simonsen', 

In [15]:
third_chance_outfits_df = pd.concat([third_chance_outfits_df, brand_df, material_df], axis=1)
#third_chance_outfits_df = pd.concat([third_chance_outfits_df, material_df], axis=1)

In [16]:
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import MinMaxScaler

brand_columns = [col for col in third_chance_outfits_df.columns if col.startswith("brand_")]
material_columns = [col for col in third_chance_outfits_df.columns if col.startswith("material_")]
LR_COLUMNS = ["condition_num", "rentals_count", "retailPrice"] + brand_columns + material_columns # "tc_price"
PREDICTION_COLUMN = ["time_diff"]
SEED = 44

train_df, test_df = train_test_split(third_chance_outfits_df, test_size=0.2, random_state=SEED)
train_df = train_df[LR_COLUMNS + PREDICTION_COLUMN].copy().fillna(0)
test_df = test_df[LR_COLUMNS + PREDICTION_COLUMN].copy().fillna(0)
train_df["time_diff"] = train_df["time_diff"].dt.days
test_df["time_diff"] = test_df["time_diff"].dt.days

min_max_scaler = MinMaxScaler()
train_df[LR_COLUMNS] = min_max_scaler.fit_transform(train_df[LR_COLUMNS])
test_df[LR_COLUMNS] = min_max_scaler.transform(test_df[LR_COLUMNS])

train_x, train_y = train_df[LR_COLUMNS].values, train_df[PREDICTION_COLUMN].values.ravel()

model = LinearRegression()
#model = LogisticRegression(max_iter=10**3)
#model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=SEED)
model.fit(train_x, train_y)

LinearRegression()

In [17]:
test_x = test_df[LR_COLUMNS].values

test_df["predicted_time_diff"] = model.predict(test_x)
test_df["difference_from_predicted"] = abs(test_df["time_diff"] - test_df["predicted_time_diff"])
test_df["difference_from_predicted"].describe()

count     197.000000
mean      337.343591
std       321.649065
min         0.253143
25%        93.218074
50%       230.610497
75%       473.060918
max      1498.941416
Name: difference_from_predicted, dtype: float64

In [18]:
test_df["difference_from_predicted"].describe()

count     197.000000
mean      337.343591
std       321.649065
min         0.253143
25%        93.218074
50%       230.610497
75%       473.060918
max      1498.941416
Name: difference_from_predicted, dtype: float64

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size_2, output_size):
        super(FCN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size_2)
        self.layer3 = nn.Linear(hidden_size_2, output_size)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

input_size = train_x.shape[1]
y_scaler = MinMaxScaler()
train_y_normalized = y_scaler.fit_transform(train_y.reshape(-1, 1))

train_x_torch = torch.tensor(train_x, dtype=torch.float32)
train_y_torch = torch.tensor(train_y_normalized, dtype=torch.float32).squeeze()

train_data = TensorDataset(train_x_torch, train_y_torch)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Adjust batch size as needed

input_size = train_x.shape[1]
HIDDEN_SIZE_1 = 100
HIDDEN_SIZE_2 = 100
OUTPUT_SIZE = 1

model = FCN(input_size, HIDDEN_SIZE_1, HIDDEN_SIZE_2, OUTPUT_SIZE)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate


In [53]:
EPOCHS = 10**2  # Number of epochs

for epoch in range(EPOCHS):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item()}')


Epoch 1/100, Loss: 0.059896666556596756
Epoch 11/100, Loss: 0.03520289063453674
Epoch 21/100, Loss: 0.014723986387252808
Epoch 31/100, Loss: 0.010332450270652771
Epoch 41/100, Loss: 0.003042250405997038
Epoch 51/100, Loss: 0.011534195393323898
Epoch 61/100, Loss: 0.012247275561094284
Epoch 71/100, Loss: 0.01908964477479458
Epoch 81/100, Loss: 0.006376284174621105
Epoch 91/100, Loss: 0.013443013653159142


In [60]:
# Predictions
test_x_torch = torch.tensor(test_x, dtype=torch.float32)
with torch.no_grad():
    model.eval()
    test_predictions = model(test_x_torch).numpy().ravel()

test_df["predicted_time_diff"] = y_scaler.inverse_transform(test_predictions.reshape(-1, 1)).ravel()
test_df["difference_from_predicted"] = abs(test_df["time_diff"] - test_df["predicted_time_diff"])
test_df["difference_from_predicted"].describe()

count     197.000000
mean      328.749361
std       352.383747
min         0.689636
25%        68.450745
50%       200.947876
75%       451.263855
max      1621.346222
Name: difference_from_predicted, dtype: float64